In [1]:
# ! pip install minio
# ! pip install pandas
# ! pip install s3fs
# ! pip install pyarrow

In [2]:
from minio import Minio
import io

In [3]:
access_key = "Q5jcNmBgOwPhrbEUfcx5"
secret_key="iD1I6ZKpkPCmuLZkIhffLmZRoy2PJklqH7XnhQXz"

In [4]:
client = Minio(
	endpoint="localhost:9000",
	access_key=access_key,
	secret_key=secret_key,
	region="sa-east-1",
 	secure=False
)

In [5]:
buckets = client.list_buckets()

In [6]:
for bucket in buckets:
    print(bucket.name, bucket.creation_date)

bronze 2024-06-09 10:01:00.069000+00:00
gold 2024-06-09 10:01:00.069000+00:00
logs 2024-06-09 10:01:00.070000+00:00
silver 2024-06-09 10:01:00.069000+00:00
staging 2024-06-09 10:01:00.069000+00:00


In [7]:
objts = client.list_objects(bucket_name="bronze", prefix=None, recursive=True)

In [8]:
for obj in objts:
    print(obj.object_name)
    

iris/iris.parquet


In [9]:
import pandas as pd
import s3fs

In [10]:
iris = pd.read_csv('https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv')

In [11]:
s3 = s3fs.S3FileSystem(
    anon=False,
    key=access_key,
    secret=secret_key,
    client_kwargs={
        'endpoint_url': 'http://localhost:9000'  # Substitua pelo endpoint do seu servidor MinIO
    }
)

In [12]:
bucket_name="bronze"
object_name="iris/iris.parquet"

In [13]:
with s3.open(f'{bucket_name}/{object_name}', 'wb') as f:
    iris.to_parquet(f, engine='pyarrow')

In [14]:
response = client.get_object(bucket_name=bucket_name, object_name=object_name)

In [15]:
df_obj = pd.read_parquet(io.BytesIO(response.read()))

In [16]:
df_obj.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
